In [ ]:
#Early Reinforcement Learning

In [1]:
# Ensure the right version of Tensorflow is installed.
!pip install tensorflow==2.1 --user

     |████████████████████████████████| 421.8 MB 29 kB/s s eta 0:00:01  |                                | 368 kB 4.6 MB/s eta 0:01:32     |█▏                              | 15.5 MB 4.6 MB/s eta 0:01:29     |████▏                           | 55.4 MB 42.5 MB/s eta 0:00:09     |█████▌                          | 71.9 MB 42.5 MB/s eta 0:00:09     |████████▏                       | 108.0 MB 40.8 MB/s eta 0:00:08     |████████████████▍               | 215.7 MB 16.2 MB/s eta 0:00:13     |████████████████████▍           | 269.4 MB 43.6 MB/s eta 0:00:04     |█████████████████████████▉      | 340.8 MB 33.1 MB/s eta 0:00:03     |██████████████████████████      | 342.8 MB 33.1 MB/s eta 0:00:03     |███████████████████████████▊    | 366.0 MB 43.7 MB/s eta 0:00:02
     |████████████████████████████████| 42 kB 1.9 MB/s  eta 0:00:01
     |████████████████████████████████| 57 kB 6.5 MB/s  eta 0:00:01
     |████████████████████████████████| 1.3 MB 23.8 MB/s eta 0:00:01
     |████████████████████████████

In [2]:
!pip install gym==0.12.5 --user

     |████████████████████████████████| 1.5 MB 6.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 28.7 MB/s eta 0:00:01
  Created wheel for gym: filename=gym-0.12.5-py3-none-any.whl size=1613801 sha256=2a20a651294b02df3d27a58411235860c40a80eba36cb2b17afbc8db787d3e7a
  Stored in directory: /home/jovyan/.cache/pip/wheels/98/2c/77/ae8eecab581ed913086cc59b17f69c54eaca71b6ba848fd2e9
Successfully built gym


In [ ]:
'''here are four methods from Gym that are going to be useful to us in order to save the gumdrop.

make allows us to build the environment or game that we can pass actions to
reset will reset an environment to it's starting configuration and return the state of the player
render displays the environment for human eyes
step takes an action and returns the player's next state.
Let's make, reset, and render the game. The output is an ANSI string with the following characters:

S for starting point
F for frozen
H for hole
G for goal
A red square indicates the current position
Note: Restart the kernel if the above libraries needed to be installed'''

In [1]:
import gym
import numpy as np
import random
env = gym.make('FrozenLake-v0', is_slippery=False)
state = env.reset()
env.render()


SFFF
FHFH
FFFH
HFFG


In [2]:
print(state)

0


In [3]:
#a simple print function to let us know whether it's game won, game over, or game on.
def print_state(state, done):
    statement = "Still Alive!"
    if done:
        statement = "Cocoa Time!" if state == 15 else "Game Over!" 
    print(state, "-", statement)

In [20]:
#0 left
#1 down
#2 right
#3 up

# Uncomment to reset the game
#env.reset()
action = 2  # Change me, please!
state, _, done, _ = env.step(action)
env.render()
print_state(state, done)

  (Right)
SFFF
FHFH
FFFH
HFFG
15 - Cocoa Time!


In [ ]:
'''Were you able to reach the hot chocolate? If so, great job!
There are multiple paths through the maze. One solution is [1, 1, 2, 2, 1, 2].
Let's loop through our actions in order to get used to interacting with the environment programmatically.'''

In [21]:
def play_game(actions):
    state = env.reset()
    step = 0
    done = False

    while not done and step < len(actions):
        action = actions[step]
        state, _, done, _ = env.step(action)
        env.render()
        step += 1
        print_state(state, done)
        
actions = [1, 1, 2, 2, 1, 2]  # Replace with your favorite path.
play_game(actions)

  (Down)
SFFF
FHFH
FFFH
HFFG
4 - Still Alive!
  (Down)
SFFF
FHFH
FFFH
HFFG
8 - Still Alive!
  (Right)
SFFF
FHFH
FFFH
HFFG
9 - Still Alive!
  (Right)
SFFF
FHFH
FFFH
HFFG
10 - Still Alive!
  (Down)
SFFF
FHFH
FFFH
HFFG
14 - Still Alive!
  (Right)
SFFF
FHFH
FFFH
HFFG
15 - Cocoa Time!


In [ ]:
#Nice, so we know how to get through the maze, but how do we teach that to the gumdrop? 
#It's just some bytes in an android phone. It doesn't have our human insight.

In [ ]:
#VALUE ITERATION
#Let's program it out and see it in action! We'll set up an array representing the lake with -1 as the holes, and 1 as the goal. 
#Then, we'll set up an array of zeros to start our iteration.

In [22]:
LAKE = np.array([[0,  0,  0,  0],
                 [0, -1,  0, -1],
                 [0,  0,  0, -1],
                 [-1, 0,  0,  1]])
LAKE_WIDTH = len(LAKE[0])
LAKE_HEIGHT = len(LAKE)

DISCOUNT = .9  # Change me to be a value between 0 and 1.
current_values = np.zeros_like(LAKE)

In [ ]:
'''
The Gym environment class has a handy property for finding the number of states in an environment called observation_space.
In our case, there a 16 integer states, so it will label it as "Discrete". 
Similarly, action_space will tell us how many actions are available to the agent.

Let's take advantage of these to make our code portable between different lakes sizes.'''

In [23]:
print("env.observation_space -", env.observation_space)
print("env.observation_space.n -", env.observation_space.n)
print("env.action_space -", env.action_space)
print("env.action_space.n -", env.action_space.n)

STATE_SPACE = env.observation_space.n
ACTION_SPACE = env.action_space.n
STATE_RANGE = range(STATE_SPACE)
ACTION_RANGE = range(ACTION_SPACE)

env.observation_space - Discrete(16)
env.observation_space.n - 16
env.action_space - Discrete(4)
env.action_space.n - 4


In [24]:
def get_neighbor_value(state_x, state_y, values, action):
    """Returns the value of a state's neighbor.
    
    Args:
        state_x (int): The state's horizontal position, 0 is the lake's left.
        state_y (int): The state's vertical position, 0 is the lake's top.
        values (float array): The current iteration's state values.
        policy (int): Which action to check the value for.
        
    Returns:
        The corresponding action's value.
    """
    left = [state_y, state_x-1]
    down = [state_y+1, state_x]
    right = [state_y, state_x+1]
    up = [state_y-1, state_x]
    actions = [left, down, right, up]

    direction = actions[action]
    check_x = direction[1]
    check_y = direction[0]
        
    is_boulder = check_y < 0 or check_y >= LAKE_HEIGHT \
        or check_x < 0 or check_x >= LAKE_WIDTH
    
    value = values[state_y, state_x]
    if not is_boulder:
        value = values[check_y, check_x]
        
    return value

In [ ]:
'''
But this doesn't find the best action, and the gumdrop is going to need that if it wants to greedily get off the lake.
The `get_max_neighbor` function we've defined below takes a number corresponding to a cell as `state_number` 
and the same value mapping as `get_neighbor_value`.
'''

In [25]:
def get_state_coordinates(state_number):
    state_x = state_number % LAKE_WIDTH
    state_y = state_number // LAKE_HEIGHT
    return state_x, state_y

def get_max_neighbor(state_number, values):
    """Finds the maximum valued neighbor for a given state.
    
    Args:
        state_number (int): the state to find the max neighbor for
        state_values (float array): the respective value of each state for
            each cell of the lake.
    
    Returns:
        max_value (float): the value of the maximum neighbor.
        policy (int): the action to take to move towards the maximum neighbor.
    """
    state_x, state_y = get_state_coordinates(state_number)
    
    # No policy or best value yet
    best_policy = -1
    max_value = -np.inf

    # If the cell has something other than 0, it's a terminal state.
    if LAKE[state_y, state_x]:
        return LAKE[state_y, state_x], best_policy
    
    for action in ACTION_RANGE:
        neighbor_value = get_neighbor_value(state_x, state_y, values, action)
        if neighbor_value > max_value:
            max_value = neighbor_value
            best_policy = action
        
    return max_value, best_policy

In [ ]:
'''
Now, let's write our value iteration code. 
We'll write a function that comes out one step of the iteration by checking each state and finding its maximum neighbor. 
The values will be reshaped so that it's in the form of the lake, but the policy will stay as a list of ints. 
This way, when Gym returns a state,
all we need to do is look at the corresponding index in the policy list to tell our agent where to go.
'''

In [26]:
def iterate_value(current_values):
    """Finds the future state values for an array of current states.
    
    Args:
        current_values (int array): the value of current states.

    Returns:
        next_values (int array): The value of states based on future states.
        next_policies (int array): The recommended action to take in a state.
    """
    next_values = []
    next_policies = []

    for state in STATE_RANGE:
        value, policy = get_max_neighbor(state, current_values)
        next_values.append(value)
        next_policies.append(policy)
    
    next_values = np.array(next_values).reshape((LAKE_HEIGHT, LAKE_WIDTH))
    return next_values, next_policies

next_values, next_policies = iterate_value(current_values)

In [27]:
next_values

array([[ 0,  0,  0,  0],
       [ 0, -1,  0, -1],
       [ 0,  0,  0, -1],
       [-1,  0,  0,  1]])

In [28]:
np.array(next_policies).reshape((LAKE_HEIGHT ,LAKE_WIDTH))

array([[ 0,  0,  0,  0],
       [ 0, -1,  0, -1],
       [ 0,  0,  0, -1],
       [-1,  0,  0, -1]])

In [29]:
current_values = DISCOUNT * next_values
current_values

array([[ 0. ,  0. ,  0. ,  0. ],
       [ 0. , -0.9,  0. , -0.9],
       [ 0. ,  0. ,  0. , -0.9],
       [-0.9,  0. ,  0. ,  0.9]])

In [38]:
next_values, next_policies = iterate_value(current_values)
print("Value")
print(next_values)
print("Policy")
print(np.array(next_policies).reshape((4,4)))
current_values = DISCOUNT * next_values

Value
[[ 0.531441  0.59049   0.6561    0.59049 ]
 [ 0.59049  -1.        0.729    -1.      ]
 [ 0.6561    0.729     0.81     -1.      ]
 [-1.        0.81      0.9       1.      ]]
Policy
[[ 1  2  1  0]
 [ 1 -1  1 -1]
 [ 2  1  1 -1]
 [-1  2  2 -1]]


In [39]:
def play_game(policy):
    state = env.reset()
    step = 0
    done = False

    while not done:
        action = policy[state]  # This line is new.
        state, _, done, _ = env.step(action)
        env.render()
        step += 1
        print_state(state, done)

play_game(next_policies)

  (Down)
SFFF
FHFH
FFFH
HFFG
4 - Still Alive!
  (Down)
SFFF
FHFH
FFFH
HFFG
8 - Still Alive!
  (Right)
SFFF
FHFH
FFFH
HFFG
9 - Still Alive!
  (Down)
SFFF
FHFH
FFFH
HFFG
13 - Still Alive!
  (Right)
SFFF
FHFH
FFFH
HFFG
14 - Still Alive!
  (Right)
SFFF
FHFH
FFFH
HFFG
15 - Cocoa Time!


In [ ]:
#Policy Iteration
#You may have noticed that the first lake was built with the parameter is_slippery=False.
#This time, we're going to switch it to True.

In [40]:
env = gym.make('FrozenLake-v0', is_slippery=True)
state = env.reset()
env.render()


SFFF
FHFH
FFFH
HFFG


In [41]:
play_game(next_policies)

  (Down)
SFFF
FHFH
FFFH
HFFG
1 - Still Alive!
  (Right)
SFFF
FHFH
FFFH
HFFG
5 - Game Over!


In [42]:
def find_future_values(current_values, current_policies):
    """Finds the next set of future values based on the current policy."""
    next_values = []

    for state in STATE_RANGE:
        current_policy = current_policies[state]
        state_x, state_y = get_state_coordinates(state)

        # If the cell has something other than 0, it's a terminal state.
        value = LAKE[state_y, state_x]
        if not value:
            value = get_neighbor_value(
                state_x, state_y, current_values, current_policy)
        next_values.append(value)

    return np.array(next_values).reshape((LAKE_HEIGHT, LAKE_WIDTH))

In [43]:
def find_best_policy(next_values):
    """Finds the best policy given a value mapping."""
    next_policies = []
    for state in STATE_RANGE:
        state_x, state_y = get_state_coordinates(state)

        # No policy or best value yet
        max_value = -np.inf
        best_policy = -1

        if not LAKE[state_y, state_x]:
            for policy in ACTION_RANGE:
                neighbor_value = get_neighbor_value(
                    state_x, state_y, next_values, policy)
                if neighbor_value > max_value:
                    max_value = neighbor_value
                    best_policy = policy
                
        next_policies.append(best_policy)
    return next_policies

In [44]:
def iterate_policy(current_values, current_policies):
    """Finds the future state values for an array of current states.
    
    Args:
        current_values (int array): the value of current states.
        current_policies (int array): a list where each cell is the recommended
            action for the state matching its index.

    Returns:
        next_values (int array): The value of states based on future states.
        next_policies (int array): The recommended action to take in a state.
    """
    next_values = find_future_values(current_values, current_policies)
    next_policies = find_best_policy(next_values)
    return next_values, next_policies

In [45]:
def get_locations(state_x, state_y, policy):
    left = [state_y, state_x-1]
    down = [state_y+1, state_x]
    right = [state_y, state_x+1]
    up = [state_y-1, state_x]
    directions = [left, down, right, up]
    num_actions = len(directions)

    gumdrop_right = (policy - 1) % num_actions
    gumdrop_left = (policy + 1) % num_actions
    locations = [gumdrop_left, policy, gumdrop_right]
    return [directions[location] for location in locations]

In [46]:
def get_neighbor_value(state_x, state_y, values, policy):
    """Returns the value of a state's neighbor.
    
    Args:
        state_x (int): The state's horizontal position, 0 is the lake's left.
        state_y (int): The state's vertical position, 0 is the lake's top.
        values (float array): The current iteration's state values.
        policy (int): Which action to check the value for.
        
    Returns:
        The corresponding action's value.
    """
    locations = get_locations(state_x, state_y, policy)
    location_chance = 1.0 / len(locations)
    total_value = 0

    for location in locations:
        check_x = location[1]
        check_y = location[0]

        is_boulder = check_y < 0 or check_y >= LAKE_HEIGHT \
            or check_x < 0 or check_x >= LAKE_WIDTH
    
        value = values[state_y, state_x]
        if not is_boulder:
            value = values[check_y, check_x]
        total_value += location_chance * value

    return total_value

In [47]:
current_values = np.zeros_like(LAKE)
policies = np.random.choice(ACTION_RANGE, size=STATE_SPACE)
np.array(policies).reshape((4,4))

array([[2, 3, 3, 3],
       [1, 1, 1, 0],
       [1, 2, 2, 2],
       [3, 0, 1, 0]])

In [54]:
next_values, policies = iterate_policy(current_values, policies)
print("Value")
print(next_values)
print("Policy")
print(np.array(policies).reshape((4,4)))
current_values = DISCOUNT * next_values

Value
[[ 7.29000e-04  0.00000e+00  0.00000e+00  0.00000e+00]
 [ 6.07500e-03 -1.00000e+00 -2.78778e-01 -1.00000e+00]
 [ 2.80260e-02  9.06660e-02  8.60490e-02 -1.00000e+00]
 [-1.00000e+00  2.35530e-01  5.13579e-01  1.00000e+00]]
Policy
[[ 0  3  3  3]
 [ 0 -1  0 -1]
 [ 3  1  0 -1]
 [-1  2  1 -1]]


In [55]:
play_game(policies)

  (Left)
SFFF
FHFH
FFFH
HFFG
4 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
0 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
0 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
0 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
4 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
4 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
8 - Still Alive!
  (Up)
SFFF
FHFH
FFFH
HFFG
4 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
8 - Still Alive!
  (Up)
SFFF
FHFH
FFFH
HFFG
9 - Still Alive!
  (Down)
SFFF
FHFH
FFFH
HFFG
8 - Still Alive!
  (Up)
SFFF
FHFH
FFFH
HFFG
4 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
0 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
0 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
4 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
4 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
4 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
8 - Still Alive!
  (Up)
SFFF
FHFH
FFFH
HFFG
8 - Still Alive!
  (Up)
SFFF
FHFH
FFFH
HFFG
9 - Still Alive!
  (Down)
SFFF
FHFH
FFFH
HFFG
13 - Still Alive!
  (Right)
SFFF
FHFH
FFFH
HFFG
14 - Still Al

In [56]:
new_row = np.zeros((1, env.action_space.n))
q_table = np.copy(new_row)
q_map = {0: 0}

def print_q(q_table, q_map):
    print("mapping")
    print(q_map)
    print("q_table")
    print(q_table)

print_q(q_table, q_map)

mapping
{0: 0}
q_table
[[0. 0. 0. 0.]]


In [57]:
def get_action(q_map, q_table, state_row, random_rate):
    """Find max-valued actions and randomly select from them."""
    if random.random() < random_rate:
        return random.randint(0, ACTION_SPACE-1)

    action_values = q_table[state_row]
    max_indexes = np.argwhere(action_values == action_values.max())
    max_indexes = np.squeeze(max_indexes, axis=-1)
    action = np.random.choice(max_indexes)
    return action

In [58]:
def update_q(q_table, new_state_row, reward, old_value):
    """Returns an updated Q-value based on the Bellman Equation."""
    learning_rate = .1  # Change to be between 0 and 1.
    future_value = reward + DISCOUNT * np.max(q_table[new_state_row])
    return old_value + learning_rate * (future_value - old_value)

In [59]:
def play_game(q_table, q_map, random_rate, render=False):
    state = env.reset()
    step = 0
    done = False

    while not done:
        state_row = q_map[state]
        action = get_action(q_map, q_table, state_row, random_rate)
        new_state, _, done, _ = env.step(action)

        #Add new state to table and mapping if it isn't there already.
        if new_state not in q_map:
            q_map[new_state] = len(q_table)
            q_table = np.append(q_table, new_row, axis=0)
        new_state_row = q_map[new_state]

        reward = -.01  #Encourage exploration.
        if done:
            reward = 1 if new_state == 15 else -1
        current_q = q_table[state_row, action]
        q_table[state_row, action] = update_q(
            q_table, new_state_row, reward, current_q)

        step += 1
        if render:
            env.render()
            print_state(new_state, done)
        state = new_state
        
    return q_table, q_map

In [60]:
# Run to refresh the q_table.
random_rate = 1
q_table = np.copy(new_row)
q_map = {0: 0}

In [61]:
q_table, q_map = play_game(q_table, q_map, random_rate, render=True)
print_q(q_table, q_map)

  (Down)
SFFF
FHFH
FFFH
HFFG
1 - Still Alive!
  (Up)
SFFF
FHFH
FFFH
HFFG
1 - Still Alive!
  (Down)
SFFF
FHFH
FFFH
HFFG
0 - Still Alive!
  (Up)
SFFF
FHFH
FFFH
HFFG
1 - Still Alive!
  (Right)
SFFF
FHFH
FFFH
HFFG
5 - Game Over!
mapping
{0: 0, 1: 1, 5: 2}
q_table
[[ 0.    -0.001  0.    -0.001]
 [ 0.    -0.001 -0.1   -0.001]
 [ 0.     0.     0.     0.   ]]


In [62]:
for _ in range(1000):
    q_table, q_map = play_game(q_table, q_map, random_rate)
    random_rate = random_rate * .99
print_q(q_table, q_map)
random_rate

mapping
{0: 0, 1: 1, 5: 2, 4: 3, 2: 4, 6: 5, 10: 6, 7: 7, 3: 8, 8: 9, 12: 10, 9: 11, 14: 12, 13: 13, 11: 14, 15: 15}
q_table
[[-0.03262936 -0.14532978 -0.14327662 -0.21812393]
 [-0.42297708 -0.51350574 -0.28919996 -0.06906911]
 [ 0.          0.          0.          0.        ]
 [-0.00625558 -0.2558071  -0.25693556 -0.32065619]
 [-0.2101821  -0.22420294 -0.22987722 -0.09385813]
 [-0.62798693 -0.62307313 -0.22250468 -0.64236568]
 [ 0.24932124 -0.20599133 -0.59662755 -0.38019376]
 [ 0.          0.          0.          0.        ]
 [-0.51916707 -0.48126436 -0.24148579 -0.102249  ]
 [-0.44933207 -0.4603394  -0.50110119  0.06121349]
 [ 0.          0.          0.          0.        ]
 [-0.25816058  0.1987019  -0.16695667 -0.14403122]
 [ 0.027892    0.75367638  0.09378429  0.27825301]
 [-0.27781368 -0.15064756  0.36112681 -0.17677364]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]]


4.317124741065784e-05

In [63]:
q_table, q_map = play_game(q_table, q_map, 0, render=True)

  (Left)
SFFF
FHFH
FFFH
HFFG
0 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
0 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
0 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
4 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
4 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
8 - Still Alive!
  (Up)
SFFF
FHFH
FFFH
HFFG
8 - Still Alive!
  (Up)
SFFF
FHFH
FFFH
HFFG
4 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
4 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
0 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
0 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
0 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
0 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
0 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
0 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
0 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
0 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
0 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
0 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
0 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
4 - Still Alive!
  (Left)
SFFF
FHFH
FFFH
HFFG
0 - Still